In [1]:
import pandas as pd
from openai import OpenAI
client = OpenAI()


In [10]:
(0.6+0.5+3)/5


0.82

In [2]:
prompt = """You are provided with an excerpt from a scientific publication. 
This excerpt claimed to provide evidence about relationship between a molecular feature and a property.
You must evaluate if this excerpt relevant to the feature and task and a valid citation?

Excerpt: {excerpt} 
Feature context: {feature}
property: {task}

Important: The feature is not direectly mentioned. You will have to infer it from the featue context.
It contains unwnated information. Please ignore it. For example if feature context is "New label: metals fraction zero" then the correct feature is "metals fraction zero" and not "New label: metals fraction zero"
Please answer Yes or No.
"""

In [3]:
tasks = ["OMS","PLD","TOX","SOL","UFL"]
titles = ["presence of open metal sites in MOFs",
          "pore limirng diameter of MOFs",
          "toxicity of small molecules",
          "solubility of small molecules",
          "upper flammability limit of small molecules"]
datapath = "data_rd3"
paper_nle = [3,4,3,1,1]

#for task, i in zip(tasks,paper_nle):
#    pd.read_csv(f"{datapath}/{task}_nle.csv").to_csv(f"{datapath}/{task}_nle_{i}.csv",index=False)

In [5]:
def get_relevance(task, title, nle_id):

    supporting = f"{datapath}/{task}/supporting_evidences_{nle_id}.csv"
    relevance = []
    df = pd.read_csv(supporting)
    for index, row in df.iterrows():    
        ft = row.feature
        query = prompt.format(excerpt=row.summary, feature=ft, task=title)
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a evaluating the relevance of a scientific citation."},
                {
                    "role": "user",
                    "content": query
                }
            ]
        )
        response = completion.choices[0].message.content
        relevance.append(response)

    df["relevance"] = relevance
    return df


In [6]:
df_OMS = get_relevance(tasks[0], titles[0], paper_nle[0])


In [8]:
df_OMS.head()

,feature,original,summary,reference,relevance
0,New label: metals fraction zero,27–30The formation or presence of open metal s...,The text discusses the concept of open metal s...,"REFERENCE:(U¨lku¨ Ko¨kçam-Demir, Anna Goldman,...",Yes
1,New label: metals fraction zero,They found that the redox properties of metal ...,The study suggests that the redox properties o...,"REFERENCE:(Zhangjing Zhang, Zi-Zhu Yao, Shengc...",No.
2,New label: metals fraction zero,to the fraction of open-metal sites estimated ...,The text discusses a new method for estimating...,"REFERENCE:(Jacklyn N. Hall, Praveen Bollini,20...",Yes
3,New label: metals fraction zero,CO2Dynamics in Pure and Mixed-Metal MOFs with ...,The study investigates the dynamics of CO2 in ...,"REFERENCE:(Robert M. Marti, Joshua D. Howe, Co...",Yes
4,New label: metals fraction zero,"larger than Ni2+, it often forms distorted oct...",The text discusses the role of metal ions in a...,"REFERENCE:(Wei Zhou, Hui Wu, Taner Yildirim,20...",No


In [9]:
#count number of yes in relevance column
df_OMS["relevance"].value_counts()


relevance
No.    6
No     6
Yes    3
Name: count, dtype: int64

In [24]:
ft = df.iloc[0].feature
query = prompt.format(excerpt=df.iloc[0].summary, feature=ft, task="presence of open metal sites in MOFs")
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a evaluating the relevance of a scientific citation."},
        {
            "role": "user",
            "content": query
        }
    ]
)

print(completion.choices[0].message.content)

No
